In [11]:
%load_ext autoreload

In [12]:
%autoreload 2

In [1]:
import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk', style='ticks',
        color_codes=True, rc={'legend.frameon': False})

%matplotlib inline

In [2]:
!nvidia-smi

Wed Aug 19 10:45:05 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 440.82       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro GV100        Off  | 00000000:37:00.0 Off |                  Off |
| 32%   44C    P2    36W / 250W |   3020MiB / 32508MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
import nfp

In [6]:
from tensorflow.keras import layers
from preprocess_inputs import preprocessor
preprocessor.from_json('tfrecords/preprocessor.json')

from loss import AtomInfMask, KLWithLogits

def parse_example(example):
    parsed = tf.io.parse_single_example(example, features={
        **preprocessor.tfrecord_features,
        **{'spin': tf.io.FixedLenFeature([], dtype=tf.string)}})

    # All of the array preprocessor features are serialized integer arrays
    for key, val in preprocessor.tfrecord_features.items():
        if val.dtype == tf.string:
            parsed[key] = tf.io.parse_tensor(
                parsed[key], out_type=preprocessor.output_types[key])
    
    # Pop out the prediction target from the stored dictionary as a seperate input
    parsed['spin'] = tf.io.parse_tensor(parsed['spin'], out_type=tf.float64)
    
    spin = parsed.pop('spin')
    
    return parsed, spin

max_atoms = 80
max_bonds = 100
batch_size = 128

# Here, we have to add the prediction target padding onto the input padding
padded_shapes = (preprocessor.padded_shapes(max_atoms=None, max_bonds=None), [None])

padding_values = (preprocessor.padding_values,
                  tf.constant(np.nan, dtype=tf.float64))

num_train = len(np.load('split.npz', allow_pickle=True)['train'])

train_dataset = tf.data.TFRecordDataset('tfrecords/train.tfrecord.gz', compression_type='GZIP')\
    .map(parse_example, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
    .cache().shuffle(buffer_size=num_train).repeat()\
    .padded_batch(batch_size=batch_size,
                  padded_shapes=padded_shapes,
                  padding_values=padding_values)\
    .prefetch(tf.data.experimental.AUTOTUNE)

In [22]:
from model import build_embedding_model

atom_embedding_model = build_embedding_model(preprocessor,
                                             atom_features=128,
                                             num_messages=6,
                                             num_heads=8,
                                             name='atom_embedding_model')

n_atom = layers.Input(shape=[], dtype=tf.int64, name='n_atom')
atom_class = layers.Input(shape=[None], dtype=tf.int64, name='atom')
bond_class = layers.Input(shape=[None], dtype=tf.int64, name='bond')
connectivity = layers.Input(shape=[None, 2], dtype=tf.int64, name='connectivity')

input_tensors = [atom_class, bond_class, connectivity, n_atom]

atom_state = atom_embedding_model(input_tensors)

atom_mean = layers.Embedding(preprocessor.atom_classes, 1,
                             name='atom_mean', mask_zero=True)(atom_class)

atom_pred = layers.Dense(1)(atom_state)
atom_pred = layers.Add()([atom_pred, atom_mean])
atom_pred = AtomInfMask()(atom_pred)

model = tf.keras.Model(input_tensors, atom_pred)

learning_rate = tf.keras.optimizers.schedules.InverseTimeDecay(1E-4, 1, 1E-5)
model.compile(loss=KLWithLogits(), optimizer=tf.keras.optimizers.Adam(learning_rate))
model.summary()

In [23]:
model.fit(train_dataset,
          steps_per_epoch=100,
          epochs=5,
          verbose=1)

Epoch 1/5
100/100 [==============================] - 4s 43ms/step - loss: 0.2782
Epoch 2/5
100/100 [==============================] - 4s 39ms/step - loss: 0.1241
Epoch 3/5
100/100 [==============================] - 4s 39ms/step - loss: 0.1022
Epoch 4/5
100/100 [==============================] - 4s 41ms/step - loss: 0.0929
Epoch 5/5
100/100 [==============================] - 4s 42ms/step - loss: 0.0853


In [24]:
model.save('test.hdf5')

In [26]:
del model

In [28]:
from layers import EdgeUpdate, NodeUpdate, GlobalUpdate

model = tf.keras.models.load_model('test.hdf5', custom_objects={
    **nfp.custom_objects, **{'AtomInfMask': AtomInfMask, 'KLWithLogits': KLWithLogits,
                             'GlobalUpdate': GlobalUpdate, 'EdgeUpdate': EdgeUpdate, 'NodeUpdate': NodeUpdate}})

ValueError: Unknown layer: NodeUpdate